In [1]:
#CNN 구현하여 mnist데이터셋
#convolution의 목적은 계층적으로 인식할 수 있게 함 
#다양한 필터를 적용하여 윤곽선,질감,털 등 각종 특징을 추출할 수 있다.
#하지만 필터를 사람이 만들기엔 너무 비효율적이라서 신경망으로 만듬
#CNN모델은 convolution 계층, pooling 계층, 특징들을 모아 최종 분류하는 일반적인 인공신경망 계층으로 구성됨
#컨볼루션 게층은 이미함지의 특징을 추출하는 열할을 함
#풀링 계층은 필터를 거친 여러 특징 중 가장 중요한 특징 하나를 고르게됨 -> 이미지 차원 감소


In [2]:
#필터가 움직이는 칸수를 조정해주는 것이 stride
#stride 가 커질수록 출력되는 텐서의 크기가 작아짐
#컨볼루션을 거쳐 만들어진 새로운 이미지는 feature map이라고 함
#feature map이 클수록 학습이 어렵고 과적합의 위험이 증가함

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')

EPOCHS = 40
BATCH_SIZE = 64

In [4]:
train_loader = torch.utils.data.DataLoader(datasets.MNIST('./.data',train = True,download=True,transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307),(0.3081))])),batch_size=BATCH_SIZE,shuffle=True)
test_loader = torch.utils.data.DataLoader(datasets.MNIST('./.data',train = False,download=True,transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307),(0.3081))])),batch_size=BATCH_SIZE,shuffle=True)

#데이터셋을 불러오고 transform을 이용하여 정규화를 진행함
#만들 CNN모델은 커널이 5*5크기이고 컨볼루션 계층은 2개임


In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(1,10,kernel_size=5)
        self.conv2 = nn.Conv2d(10,20,kernel_size=5)
        self.drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320,50)
        self.fc2 = nn.Linear(50,10)

#모델의 학습 모듈을 정의해줌
    #다음은 입력부터 출력까지 데이터가 지나가는 길을 만들어야함
    def forward(self,x):
        x = F.relu(F.max_pool2d(self.conv1(x),2))
        x = F.relu(F.max_pool2d(self.conv2(x),2)) #컨볼루션과 풀링을 지난 뒤 relu 활성화함수를 통과함
    #이제 x는 컨볼루션 계층 2개를 거친 특징 맵이 됨
    #현재 2차원인 x를 바로 인공신경망에 넣을 수 없음(인공신경망은 1차원을 받게 되어있기 때문)
        x = x.view(-1,320) # view함수는 2차원을 1차원으로 펴주는 역할을 하고 -1은 남는차원 모두, 320은 x가 가진 원소개수를 써줌
        x = F.relu(self.fc1(x))
        x = self.drop(x)
        x = self.fc2(x)
        return F.log_softmax(x,dim=1)

In [8]:
model = Net().to(DEVICE)
optimizer = optim.SGD(model.parameters(),lr=0.01,momentum=0.5)#최적화함수
def train(model,train_loader,optimizer,epoch):
    model.train()
    for batch_idx,(data,target) in enumerate(train_loader):
        data,target = data.to(DEVICE),target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output,target)
        loss.backward()
        optimizer.step()
        if batch_idx%200 == 0:
            print('train epoch',epoch,'loss',loss)
def evaluate(model,test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data,target in test_loader:
            data,target = data.to(DEVICE),target.to(DEVICE)
            output = model(data)
            test_loss += F.cross_entropy(output,target,reduction = 'sum').item()
            pred = output.max(1,keepdim = True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    test_acc = 100*correct/len(test_loader.dataset)
    return test_loss,test_acc

for epoch in range(EPOCHS):
    train(model,train_loader,optimizer,epoch)
    test_loss,test_acc = evaluate(model,test_loader)
    print('epoch',epoch,'loss',test_loss,'acc',test_acc)

/home/hyunskki/anaconda3/lib/python3.9/site-packages/torch/nn/functional.py:1331: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


train epoch 0 loss tensor(2.3477, device='cuda:0', grad_fn=<NllLossBackward0>)
train epoch 0 loss tensor(0.7509, device='cuda:0', grad_fn=<NllLossBackward0>)
train epoch 0 loss tensor(0.6407, device='cuda:0', grad_fn=<NllLossBackward0>)
train epoch 0 loss tensor(0.4026, device='cuda:0', grad_fn=<NllLossBackward0>)
train epoch 0 loss tensor(0.4393, device='cuda:0', grad_fn=<NllLossBackward0>)
epoch 0 loss 0.17391465904712677 acc 94.93
train epoch 1 loss tensor(0.2287, device='cuda:0', grad_fn=<NllLossBackward0>)
train epoch 1 loss tensor(0.2609, device='cuda:0', grad_fn=<NllLossBackward0>)
train epoch 1 loss tensor(0.4958, device='cuda:0', grad_fn=<NllLossBackward0>)
train epoch 1 loss tensor(0.2799, device='cuda:0', grad_fn=<NllLossBackward0>)
train epoch 1 loss tensor(0.1107, device='cuda:0', grad_fn=<NllLossBackward0>)
epoch 1 loss 0.09281325154900551 acc 96.85
train epoch 2 loss tensor(0.2884, device='cuda:0', grad_fn=<NllLossBackward0>)
train epoch 2 loss tensor(0.1673, device='cud

train epoch 18 loss tensor(0.0056, device='cuda:0', grad_fn=<NllLossBackward0>)
epoch 18 loss 0.028746916813380086 acc 99.11
train epoch 19 loss tensor(0.0633, device='cuda:0', grad_fn=<NllLossBackward0>)
train epoch 19 loss tensor(0.0412, device='cuda:0', grad_fn=<NllLossBackward0>)
train epoch 19 loss tensor(0.1838, device='cuda:0', grad_fn=<NllLossBackward0>)
train epoch 19 loss tensor(0.0485, device='cuda:0', grad_fn=<NllLossBackward0>)
train epoch 19 loss tensor(0.0191, device='cuda:0', grad_fn=<NllLossBackward0>)
epoch 19 loss 0.02810648284088529 acc 99.05
train epoch 20 loss tensor(0.0866, device='cuda:0', grad_fn=<NllLossBackward0>)
train epoch 20 loss tensor(0.0529, device='cuda:0', grad_fn=<NllLossBackward0>)
train epoch 20 loss tensor(0.0147, device='cuda:0', grad_fn=<NllLossBackward0>)
train epoch 20 loss tensor(0.0151, device='cuda:0', grad_fn=<NllLossBackward0>)
train epoch 20 loss tensor(0.0933, device='cuda:0', grad_fn=<NllLossBackward0>)
epoch 20 loss 0.030697220949549

train epoch 37 loss tensor(0.1407, device='cuda:0', grad_fn=<NllLossBackward0>)
train epoch 37 loss tensor(0.0110, device='cuda:0', grad_fn=<NllLossBackward0>)
train epoch 37 loss tensor(0.2653, device='cuda:0', grad_fn=<NllLossBackward0>)
train epoch 37 loss tensor(0.0884, device='cuda:0', grad_fn=<NllLossBackward0>)
epoch 37 loss 0.028296158336452208 acc 99.18
train epoch 38 loss tensor(0.1262, device='cuda:0', grad_fn=<NllLossBackward0>)
train epoch 38 loss tensor(0.0549, device='cuda:0', grad_fn=<NllLossBackward0>)
train epoch 38 loss tensor(0.0332, device='cuda:0', grad_fn=<NllLossBackward0>)
train epoch 38 loss tensor(0.0300, device='cuda:0', grad_fn=<NllLossBackward0>)
train epoch 38 loss tensor(0.0126, device='cuda:0', grad_fn=<NllLossBackward0>)
epoch 38 loss 0.026421580373853795 acc 99.11
train epoch 39 loss tensor(0.0179, device='cuda:0', grad_fn=<NllLossBackward0>)
train epoch 39 loss tensor(0.0958, device='cuda:0', grad_fn=<NllLossBackward0>)
train epoch 39 loss tensor(0.0